In [3]:
import ee
import pandas as pd
import geemap
import geopandas as gpd
import modules.tidy_tables as tidy_tables
import json
import numpy as np
import time

from modules.gee_initialize import initialize_ee 

initialize_ee()

from parameters.config_runtime import country_dataset_name
from parameters.config_output_naming import geo_id_column, geometry_area_column, plot_id_column

from parameters.config_lookups import lookup_gee_datasets

%run alt_whisp_multiband_image.ipynb

## get lists from lookup for different streams of processing
from  dataset_properties.make_processing_lists_from_lookup import buffer_stats_list,\
                                                                  presence_only_flag_list,\
                                                                  country_allocation_stats_only_list, \
                                                                  plot_stats_list, \
                                                                  decimal_place_column_list

In [ ]:
# %store -r bob
# print (bob)

In [5]:
st = time.time()

%store -r roi

stats= get_stats_fc(roi)

stats

print ('stats time:', time.time() - st, 'seconds')# get the execution time
st = time.time()

df = geemap.ee_to_pandas(stats)

prefix_columns_list =[geo_id_column,geometry_area_column,"Country"] 

df_reordered = tidy_tables.reorder_columns_by_lookup(
    df,lookup_gee_datasets,'dataset_order',"dataset_name",prefix_columns_list).drop(country_dataset_name, axis=1, errors='ignore')

df_out = df_reordered

# convert positive results values to "True" for specific columns and fix decimals
df_out[presence_only_flag_list] = np.where(df_out[presence_only_flag_list] > 0, "True", "-")

df_out[decimal_place_column_list] = df_out[decimal_place_column_list].round(decimals=0, out=None).astype(int)

#adding in index plot id column for visual cross referencing on map etc
df_out[plot_id_column] = df_out.index +1

df_out.set_index(plot_id_column, inplace = True)

# ceo_cols_for_exporting = df_out.columns # with underscores for use in ceo export

# remove underscores in column names
df_out.columns = df_out.columns.str.replace('_', ' ')

print ('table tidying time:', time.time() - st, 'seconds')# get the execution time

df_out

TypeError: 'PickleShareDB' object is not subscriptable